In [1]:
import pandas as pd
import numpy as np
import string
import random
import pickle
from tqdm import tqdm
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS
from gensim.models.doc2vec import TaggedDocument
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pyresparser import ResumeParser
import re
import csv

In [2]:
###DECLARE CV PATH###
path = '/home/ftraverso/code/francescotraverso/job_predictor/data/cv_directory/cv_rui.pdf'

###OPEN RESUME###
resume = ResumeParser(path).get_extracted_data()

###EXTRACT JOB EXPERIENCE###
JD = " ".join(resume['experience'])
JD

/home/ftraverso/.pyenv/versions/3.8.12/envs/job_predictor-env/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


'and report information. Looking for a career change to a purer Data Science role.'

In [3]:
JD

'and report information. Looking for a career change to a purer Data Science role.'

In [9]:
with open('../data/skill_packages/ESCO/marketing_skills_esco.csv', newline='') as f:
    reader = csv.reader(f)
    csv_data = list(reader)
    ds_keywords = [item for sublist in csv_data for item in sublist]

ds_skills = " ".join(ds_keywords)
#ds_skills

In [ ]:
############################
### describe your job!!! ###
############################

describe_your_job = JD + ds_skills

##################################
### get 'n' job suggeestion!!! ###
##################################

n_jobs = 10

describe_your_job

In [ ]:
# get dataframes from CSV files

df_occ_n_skills = pd.read_csv('../data/ESCO/occupations_augmented_with_skills.csv')

In [ ]:
# filter unneeded columns of of the dataframe and add needed ones

df_occ_n_skills = df_occ_n_skills.filter(items=['preferredLabel', 'description', 'skills'])
df_occ_n_skills = df_occ_n_skills.reindex(columns=['preferredLabel','description', 'skills'])
df_occ_n_skills.rename(columns={'preferredLabel': 'job_title'}, inplace=True)
df_occ_n_skills['description_input'] = 0
df_occ_n_skills['skills_input'] = 0

In [ ]:
# create description_input and skills_input, which are strings on which the model will be fit

for row, index in tqdm(df_occ_n_skills.iterrows()):
    underscored_job_title = index['job_title'].replace(" ", "_")
    this_rows_description_input = underscored_job_title + ' ' + index['description']
    this_rows_skills_input = underscored_job_title + ' ' + index['skills']
    df_occ_n_skills.iloc[row,-2] = this_rows_description_input
    df_occ_n_skills.iloc[row,-1] = this_rows_skills_input

In [ ]:
# inspect dataframe

df_occ_n_skills.head()

In [ ]:
# get entire dataframe as dataset according to Qiewi's suggestion
# concatenate the job_title:skills list to the end of df_occ_n_skills

X_all = pd.concat([df_occ_n_skills['description_input'], df_occ_n_skills['skills_input']]).reset_index(drop=True)

In [ ]:
# applying preprocessing to corpus as in Gensim tutorial, also applying Phraser

def read_corpus(corpus):
   
    # instantiate Phraser outside of the loop
    sentence_stream = [entry.split(" ") for entry in corpus]
    bigrams = Phrases(
        sentence_stream,
        min_count=5,
        threshold=5,
        connector_words=ENGLISH_CONNECTOR_WORDS
        )
    
    for i, line in enumerate(corpus):    
    
        # remove punctuation
        for punctuation in string.punctuation:        
            sentence = line.replace(punctuation, '')

        # remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = word_tokenize(sentence)
        stopword_free_tokens = [token for token in tokens if token not in stop_words]
        sentence = ' '.join(stopword_free_tokens)

        # lemmatize
        sentence = WordNetLemmatizer().lemmatize(sentence, pos='n')
        sentence = WordNetLemmatizer().lemmatize(sentence, pos='v')
        
        # get bigrams
        sent = sentence.split()

        # yield tagged final corpus
        yield TaggedDocument(bigrams[sent], [i])

all_corpus = list(read_corpus(X_all))

In [ ]:
# Preprocessing function for job descriptions

def preprocess_input(sentence, ds_insert=False, ds_insert_ratio=0.2):
    
    if ds_insert == True:

        sentence_splitted = sentence.split()

        if len(sentence_splitted) * ds_insert_ratio >= len(ds_keywords):
            print('Warning: Chosen ratio is using up all available Data Science keywords!')

        insertion_amount = int(len(sentence_splitted) * ds_insert_ratio)
        insertion_counter = 0

        for insertion in range(insertion_amount):
            if len(sentence_splitted) * ds_insert_ratio <= len(ds_keywords):
                sentence_splitted.append(ds_keywords[insertion_counter])
                insertion_counter += 1

        sentence = ' '.join(sentence_splitted)
    
    # remove punctuation
    for punctuation in string.punctuation:        
        sentence = sentence.replace(punctuation, '')
    
    # set lowercase
    sentence = sentence.lower()
    
    # remove numbers
    sentence = ''.join(char for char in sentence if not char.isdigit())
    
    # remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(sentence)
    stopword_free_tokens = [token for token in tokens if token not in stop_words]
    sentence = ' '.join(stopword_free_tokens)

    # lemmatize
    sentence = WordNetLemmatizer().lemmatize(sentence, pos='n')
    sentence = WordNetLemmatizer().lemmatize(sentence, pos='v')
    
    # split into tokens again after Lemmatizing --- this was replaced by Phraser 
    # sentence = word_tokenize(sentence)
    
    # insert data science keywords if ds_insert==True
    
    
    return sentence

In [ ]:
ds_keywords

In [ ]:
new_description = preprocess_input(describe_your_job)
# new_description

In [ ]:
# load saved BERT model and its embedding from disk

filename = '../model/bert_model.sav'
bert_model = pickle.load(open(filename, 'rb'))

filename = '../model/all_corpus_embed.sav'
all_corpus_embed = pickle.load(open(filename, 'rb'))

In [ ]:
# # run model

# # run input through the model
# new_description_embed = bert_model.encode(new_description)

# # calculate and order cosine similarity
# similarity_rank = cosine_similarity([new_description_embed], all_corpus_embed)
# similarity_rank_index = np.argsort(similarity_rank[0])[::-1]



In [ ]:
### result comparison between no DS keywords and DS keywords ###

# insert the ratio you want for the insertion of data science keywords (value between 0 and 1)
ds_ratio = 0

# preprocess
new_description_no_ds = preprocess_input(describe_your_job)
new_description_w_ds = preprocess_input(describe_your_job, ds_insert=True, ds_insert_ratio=ds_ratio)

print(new_description_no_ds)
print(new_description_w_ds)


# run input through the model
new_description_no_ds_embed = bert_model.encode(new_description_no_ds)
new_description_w_ds_embed = bert_model.encode(new_description_w_ds)

# calculate and order cosine similarity
similarity_rank_no_ds = cosine_similarity([new_description_no_ds_embed], all_corpus_embed)
similarity_rank_index_no_ds = np.argsort(similarity_rank_no_ds[0])[::-1]
similarity_rank_w_ds = cosine_similarity([new_description_w_ds_embed], all_corpus_embed)
similarity_rank_index_w_ds = np.argsort(similarity_rank_w_ds[0])[::-1]

# show results
print('~~~RESULT COMPARISON~~~ \n')

for i in range(n_jobs):
    
    if similarity_rank_index_no_ds[i] <= len(df_occ_n_skills):
        new_index = similarity_rank_index_no_ds[i]
    else:
        new_index = similarity_rank_index_no_ds[i] - len(df_occ_n_skills)

    print(f'RANK #{i+1}:')
    print('ORIGINAL: ' + df_occ_n_skills.loc[new_index]['job_title'])

    if similarity_rank_index_w_ds[i] <= len(df_occ_n_skills):
        new_index = similarity_rank_index_w_ds[i]
    else:
        new_index = similarity_rank_index_w_ds[i] - len(df_occ_n_skills)
    
    print('W/ DS KW: ' + df_occ_n_skills.loc[new_index]['job_title'])
    print('')

In [ ]:
# print(f'TEST DOCUMENT: {new_description} \n')
# print('LISTING 10 MOST SIMILAR JOB ROLES & DESCRIPTIONS \n')

# for i in range(n_jobs):
    
#     if similarity_rank_index[i] <= len(df_occ_n_skills):
#         new_index = similarity_rank_index[i]
#     else:
#         new_index = similarity_rank_index[i] - len(df_occ_n_skills)

#     print(f'RANK #{i+1}: ' + df_occ_n_skills.loc[new_index]['job_title'])
#     print(df_occ_n_skills.loc[new_index]['description'])
#     print(f'Similarity score: {round(similarity_rank[0][similarity_rank_index[i]]*100,1)} %')
#     print(f'Index in dataframe: {new_index} \n')

